# Currency Crash Prediction - Backtest Analysis

Backtest the R-Zone strategy with transaction costs and risk management

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from signal_generator import CurrencyCrashPredictor
import yaml

sns.set_style('darkgrid')
%matplotlib inline

## 1. Load Signals and Configuration

In [ ]:
with open('../config.yaml', 'r') as f:
    config = yaml.safe_load(f)

predictor = CurrencyCrashPredictor()
predictor.load_data()
signals = predictor.generate_signals()

fx_rates = predictor.fx_rates
r_zone = signals['r_zone']
monthly_returns = signals['monthly_returns']

## 2. Simple Backtest: Short on R-Zone Entry

In [ ]:
def backtest_strategy(r_zone, monthly_returns, config):
    """
    Simple backtest: Short currency on R-Zone entry, hold 6 months
    """
    position_duration = config['position_duration_months']
    max_exposure = config['max_exposure']
    
    # Track positions and returns
    positions = {}  # {(currency, entry_date): exit_date}
    trades = []
    
    for currency in r_zone.columns:
        r_zone_dates = r_zone[r_zone[currency] == 1].index
        
        for entry_date in r_zone_dates:
            # Calculate exit date (6 months later)
            exit_dates = pd.date_range(entry_date, periods=position_duration+1, freq='M')[1:]
            
            # Calculate returns over holding period
            holding_returns = monthly_returns.loc[
                monthly_returns.index.isin(exit_dates), currency
            ]
            
            if len(holding_returns) > 0:
                # Short position: profit when currency depreciates (negative return)
                cumulative_return = -holding_returns.sum()  # Negative because short
                
                # Apply transaction costs
                currency_type = 'advanced' if currency in config['currencies']['advanced'] else 'emerging'
                tc = config['transaction_costs'][currency_type] / 10000  # bps to decimal
                slippage = config['slippage'][currency_type] / 10000
                
                net_return = cumulative_return - 2 * (tc + slippage)  # Entry + exit
                
                trades.append({
                    'currency': currency,
                    'entry_date': entry_date,
                    'exit_date': exit_dates[-1] if len(exit_dates) > 0 else entry_date,
                    'gross_return': cumulative_return,
                    'net_return': net_return,
                    'holding_months': len(holding_returns)
                })
    
    return pd.DataFrame(trades)

trades_df = backtest_strategy(r_zone, monthly_returns, config)
print(f"Total Trades: {len(trades_df)}")
print(f"\nFirst 10 trades:")
print(trades_df.head(10))

## 3. Performance Metrics

In [ ]:
# Calculate performance metrics
win_rate = (trades_df['net_return'] > 0).mean()
avg_return = trades_df['net_return'].mean()
avg_win = trades_df[trades_df['net_return'] > 0]['net_return'].mean()
avg_loss = trades_df[trades_df['net_return'] < 0]['net_return'].mean()
profit_factor = abs(avg_win / avg_loss) if avg_loss != 0 else np.nan

print("=" * 50)
print("BACKTEST PERFORMANCE METRICS")
print("=" * 50)
print(f"Total Trades: {len(trades_df)}")
print(f"Win Rate: {win_rate:.2%}")
print(f"Average Return per Trade: {avg_return:.2%}")
print(f"Average Win: {avg_win:.2%}")
print(f"Average Loss: {avg_loss:.2%}")
print(f"Profit Factor: {profit_factor:.2f}")
print(f"Best Trade: {trades_df['net_return'].max():.2%}")
print(f"Worst Trade: {trades_df['net_return'].min():.2%}")
print("=" * 50)

## 4. Cumulative Returns

In [ ]:
# Calculate cumulative returns over time
trades_df_sorted = trades_df.sort_values('entry_date')
trades_df_sorted['cumulative_return'] = (1 + trades_df_sorted['net_return']).cumprod() - 1

plt.figure(figsize=(14, 6))
plt.plot(trades_df_sorted['entry_date'], 
         trades_df_sorted['cumulative_return'] * 100, 
         linewidth=2)
plt.axhline(y=0, color='r', linestyle='--', alpha=0.5)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Cumulative Return (%)', fontsize=12)
plt.title('Currency Crash Strategy - Cumulative Returns', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Returns Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
axes[0].hist(trades_df['net_return'] * 100, bins=30, 
            edgecolor='black', alpha=0.7)
axes[0].axvline(0, color='r', linestyle='--', linewidth=2)
axes[0].set_xlabel('Return (%)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Trade Returns')
axes[0].grid(True, alpha=0.3)

# Box plot by currency
trades_df.boxplot(column='net_return', by='currency', ax=axes[1])
axes[1].set_xlabel('Currency')
axes[1].set_ylabel('Net Return')
axes[1].set_title('Returns by Currency')
plt.suptitle('')

plt.tight_layout()
plt.show()

## 6. In-Sample vs Out-of-Sample Performance

In [ ]:
in_sample_end = pd.to_datetime(config['in_sample_end'])

in_sample = trades_df[trades_df['entry_date'] <= in_sample_end]
out_sample = trades_df[trades_df['entry_date'] > in_sample_end]

print("IN-SAMPLE PERFORMANCE (1999-2018):")
print(f"  Trades: {len(in_sample)}")
print(f"  Win Rate: {(in_sample['net_return'] > 0).mean():.2%}")
print(f"  Avg Return: {in_sample['net_return'].mean():.2%}")

print("\nOUT-OF-SAMPLE PERFORMANCE (2019-2023):")
print(f"  Trades: {len(out_sample)}")
print(f"  Win Rate: {(out_sample['net_return'] > 0).mean():.2%}")
print(f"  Avg Return: {out_sample['net_return'].mean():.2%}")

## 7. Risk Metrics

In [ ]:
# Calculate Sharpe ratio (assuming monthly rebalancing)
monthly_strategy_returns = trades_df.groupby('entry_date')['net_return'].mean()

sharpe_ratio = monthly_strategy_returns.mean() / monthly_strategy_returns.std() * np.sqrt(12)
max_drawdown = (monthly_strategy_returns.cumsum().cummax() - monthly_strategy_returns.cumsum()).max()

print("RISK METRICS:")
print(f"Sharpe Ratio (annualized): {sharpe_ratio:.2f}")
print(f"Max Drawdown: {max_drawdown:.2%}")
print(f"Volatility (annualized): {monthly_strategy_returns.std() * np.sqrt(12):.2%}")

## 8. Export Results

In [ ]:
# Save trades to CSV
trades_df.to_csv('../data/backtest_trades.csv', index=False)
print("Backtest results saved to data/backtest_trades.csv")